In [1]:
import pandas as pd

In [2]:
# Define files
input_file = 'homework.csv'
output_file = 'formatted.csv'
example_file = 'example.csv'

In [17]:
# Options
pd.set_option('display.max_columns', None)

## Extract

In [18]:
# Define dataframes
input_df = pd.read_csv(input_file)
example_df = pd.read_csv(example_file)

/Users/dominicsciarrino/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3398: DtypeWarning: Columns (6,7,23,43,45,86,87,91,92,122,123,124) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [19]:
# Show input file
input_df.head(1)

,item number,upc,new item,system creation date,min order qty,sales uom,wholesale ($),map ($),msrp ($),description,long description,brand,item category,item type,outdoor,item width (inches),item depth (inches),item height (inches),item diameter (inches),item weight (pounds),multi-piece dimension 1 (inches),multi-piece dimension 2 (inches),multi-piece dimension 3 (inches),multi-piece dimension 4 (inches),item materials,primary color family,item finish,item finish 1,item finish 2,item finish 3,primary image filename,url primary image,url alternate image 1,url alternate image 2,url alternate image 3,url alternate image 4,url alternate image 5,url alternate image 6,url alternate image 7,url alternate image 8,url room setting image 1,url room setting image 2,url room setting image 3,url drawing,url interactive 360 image,url animated gif,url product sheet,url instruction sheet,url marketing sheet 1,url california label (jpg),url california label (pdf),item style,item substyle,item substyle 2,item collection,licensed by,carton count,truck only,carton 1 width (inches),carton 1 length (inches),carton 1 height (inches),carton 1 weight (pounds),carton 1 volume (cubic feet),carton 2 width (inches),carton 2 length (inches),carton 2 height (inches),carton 2 weight (pounds),carton2volumecubicfeet,carton 3 width (inches),carton 3 length (inches),carton 3 height (inches),carton 3 weight (pounds),carton 3 volume (cubic feet),ada compliant,available with eef,conversion kit option,title 24 compliant,safety rating,certified damp/wet,bulb 1 count,bulb 1 wattage,bulb 1 type,bulb 1 base,bulb 1 included,bulb 2 count,bulb 2 wattage,bulb 2 type,bulb 2 base,bulb 2 included,led,total lumens,color temperature,cri,voltage,switch type,dimmable,lamp base dimensions (inches),backplate/canopy dimensions (inches),extension rods (inches),min overall height (inches),max overall height (inches),min extension (inches),max extension (inches),hcwo (inches),shade/glass description,shade/glass materials,shade/glass finish,shade/glass width,shade/glass width at top (inches),shade/glass width at bottom (inches),shade/glass height (inches),shade shape,harp/spider,cord color,cord length (inches),chain length (inches),chain price ($),replacement glass price ($),replacement crystal price ($),mirror width (inches),mirror height (inches),drawer count,drawer 1 interior dimensions (inches),drawer 2 interior dimensions (inches),drawer 3 interior dimensions (inches),furniture arm height (inches),furniture seat height (inches),furniture seat dimensions (inches),furniture weight capacity (pounds),country of origin,primary catalog,primary catalog page,related items,brand bio,helpful tips,selling point 1,selling point 2,selling point 3,selling point 4,selling point 5,selling point 6,selling point 7,selling point 8,selling point 9,selling point 10,record status
0,203,8.109370e+11,NaN,7/7/15,1,Each,228.0,304.0,$456.00,White Obelisk Table Lamp With Night Light,NaN,Dimond Lighting,Indoor Lighting,Table Lamp,No,18.0,18.0,36.0,18.0,6.0,NaN,NaN,NaN,NaN,Glass,NaN,White,White,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://s3-us-west-2.amazonaws.com/elk-assets/...,NaN,NaN,https://s3-us-west-2.amazonaws.com/elk-assets/...,https://s3-us-west-2.amazonaws.com/elk-assets/...,Modern / Contemporary,Luxe / Glam,NaN,Obelisk,NaN,1.0,No,19.0,19.0,34.0,10.0,7.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,No,NaN,UL,NaN,1.0,100.0,A21 3-Way,E26 Medium,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3-Way,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Round Hardback White Fabric Shade,Fabric,White,NaN,16.0,18.0,11.0,NaN,NaN,NaN,66.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,China,ELK Home 2019,529,NaN,Dimond Lighting is an award-winning designer a...,When choosing a table lamp it is important to ...,8-Inch wide base,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Active
